Get the number of concepts after non-concept filtering by ChatGPT：

In [ ]:
import os
import json
from tqdm import tqdm
label_path = "./data_122/m2conceptbase/concept_labels/"

label_files = os.listdir(label_path)


concept_list = []
non_concept_list = []
for label_file in tqdm(label_files, total=len(label_files)):
    # print(label_file)
    try:
        with open(os.path.join(label_path, label_file), 'r', encoding='utf-8') as f:
            concept_labels = json.load(f)
        
        concept_list.extend([concept for concept, label in concept_labels.items() if label == 1])
        non_concept_list.extend([concept for concept, label in concept_labels.items() if label == 0])
    except Exception as e:
        print(label_file)
        print(e)

100%|██████████| 5828/5828 [01:46<00:00, 54.94it/s] 


In [15]:
print(len(concept_list))
print(len(non_concept_list))
non_concept_list
with open("concepts_230606.json", 'w', encoding='utf-8') as f:
    json.dump(concept_list, f, ensure_ascii=False, indent=4)
with open("non_concepts_230606.json", 'w', encoding='utf-8') as f:
    json.dump(non_concept_list, f, ensure_ascii=False, indent=4)

139431
210192


### 概念本义缺失判断

In [ ]:
import os
import json
from tqdm import tqdm
concept_descriptions_path = "./data_122/m2conceptbase/concept_descriptions/"

letter_list = [chr(ord('A') + i) for i in range(26)]


def lack_basic_meaning(concept_descriptions):
    lack = True
    for description in concept_descriptions:
        if "《" not in description and "》" not in description and "产品" not in description:
            lack = False
    return lack

concepts_lack_basic_meaning = []

for letter in letter_list:
    dir_path = os.path.join(concept_descriptions_path, letter)
    print("processing {}...".format(dir_path))
    concept_files = os.listdir(dir_path)
    for filename in tqdm(concept_files, total=len(concept_files)):
        concept_filename = os.path.join(dir_path, filename)
        with open(concept_filename, 'r', encoding='utf-8') as f:
            concept_dict = json.load(f)
            concept_name = concept_dict["concept"]
            concept_descriptions = concept_dict["candidate_descriptions"]
        if lack_basic_meaning(concept_descriptions):
            # print("{} is lack of basic meaning.".format(concept_name))
            concepts_lack_basic_meaning.append(concept_name)
        # else:
        #     print(concept_name)

print("total {} concepts lacking basic meaning.".format(len(concepts_lack_basic_meaning)))

with open("concepts_lack_basic_meaning.json", 'w', encoding='utf-8') as f:
    json.dump(concepts_lack_basic_meaning, f, ensure_ascii=False, indent=4)
print("done.")

In [ ]:
import json
with open("concepts_lack_basic_meaning.json", 'r', encoding='utf-8') as f:
    concepts_lack_basic_meaning = json.load(f)
    
with open("./projects/m2conceptbase/utils/word_freq_train_data_230531.json", 'r', encoding='utf-8') as f:
    word_dict = json.load(f)

concepts_lack_basic_meaning_filtered = [word for word in concepts_lack_basic_meaning if word in word_dict]
len(concepts_lack_basic_meaning_filtered)
with open("concepts_lack_basic_meaning_v2.json", 'w', encoding='utf-8') as f:
    json.dump(concepts_lack_basic_meaning_filtered, f, ensure_ascii=False, indent=4)
print("done.")

done.


In [ ]:
with open("./projects/m2conceptbase/utils/concepts_230606.json", 'r', encoding='utf-8') as f:
    concepts = json.load(f)
    
with open("./projects/m2conceptbase/concepts_crawled_230608.json", 'r', encoding='utf-8') as f:
    concepts_crawled = json.load(f)
    
concepts_to_generate = set(concepts) - set(concepts_crawled) | set(concepts_lack_basic_meaning_filtered)
print(len(concepts_to_generate)) # 187154
with open("./projects/m2conceptbase/concepts_to_generate_230608.json", 'w', encoding='utf-8') as f:
    json.dump(list(concepts_to_generate), f, ensure_ascii=False, indent=4)
print("done.")

187154
done.


第一步，根据上述规则，得到缺失本义的概念。
第二步，判断概念是否在词性nw（或nz）字典中（前提是在候选概念字典中）
第三步，为这些概念利用ChatGPT生成基本描述。